# Extract Data Mailbox

In [ ]:
import win32com.client
import pandas as pd
from datetime import datetime, timedelta

# Outlook Connection
outlook = win32com.client.Dispatch("Outlook.Application").GetNamespace("MAPI")
shared_mailbox = outlook.Folders("Shared Mailbox").Folders("Inbox")

messages = shared_mailbox.Items
messages.Sort("[ReceivedTime]", True)

# Filter Date
today = datetime.today()
start_date = datetime(2025, 6, 11)
end_date = datetime(2025, 6, 20)
# Output list
data = []

# Loop
for message in messages:
    try:
        received_time = message.ReceivedTime.replace(tzinfo=None)
        if not (start_date <= received_time <= end_date):
            continue

        sender_name = message.SenderName
        subject = message.Subject
        received = received_time.strftime("%Y-%m-%d %H:%M:%S")

        # Checking All Attachment File
        for attachment in message.Attachments:
            if attachment.FileName.lower().endswith(".pdf"):
                attachment_size_kb = round(attachment.Size / 1024, 2)  # KB size
                data.append([
                    sender_name,
                    subject,
                    received,
                    attachment.FileName,
                    attachment_size_kb
                ])

    except Exception as e:
        print(f"Error: {e}")

# Create Data Frame
df = pd.DataFrame(data, columns=["Sender Name", "Subject", "Sent Time", "Attachment Name", "Attachment Size (KB)"])

# Cleaning Attachment Name (.pdf)
df["Attachment Name Clean"] = df["Attachment Name"].str.replace(".pdf", "", case=False).str.strip()

#ExtractTranform Attachment File After First "_"
def extract_invoice_after_delimiter(name):
    for delimiter in ['_', '-']:
        if delimiter in name:
            return name.split(delimiter, 1)[1].split('.')[0]
    return None

df["Attachment Number"] = df["Attachment Name Clean"].apply(extract_invoice_after_delimiter)

# Calculate Total Size of Subject Mail
size_per_subject = df.groupby("Subject")["Attachment Size (KB)"].sum().reset_index()
size_per_subject.rename(columns={"Attachment Size (KB)": "Total Size (KB) per Subject"}, inplace=True)

# Combine Dataframe
df = pd.merge(df, size_per_subject, on="Subject", how="left")


# Size on MB
df["Total Size (MB) per Subject"] = (df["Total Size (KB) per Subject"] / 1024).round(2)

# Export to Excel
output_file = f"Extract Mailbox Outlook_11-20June25.xlsx"
df.to_excel(output_file, index=False)

print(f"✅ Complete Extract Data on: {output_file}")
